In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import datetime

dataset = yf.download('TSLA', period='3y')

train_set = dataset[['Close']][:-30].fillna(method='ffill')
train_set = train_set.values.reshape(-1,1)
dataset.shape, train_set.shape

In [ ]:
sc = MinMaxScaler(feature_range=(0,1))
sc = sc.fit(train_set)
train_set_scaled = sc.transform(train_set)
train_set_scaled[:10]

In [ ]:
X=[]
y=[]
lookback = 60
forecast = 30
for i in range(lookback, len(train_set_scaled) - forecast + 1):
    X.append(train_set_scaled[i - lookback : i])
    y.append(train_set_scaled[i : i + forecast])
    
X=np.array(X)
y=np.array(y)

In [ ]:
#model

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(lookback, 1)))
model.add(LSTM(units=50))
model.add(Dense(forecast))

model.summary()

In [ ]:
#train

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X, y, epochs=100, batch_size=32)

In [ ]:
X_ = train_set_scaled[-lookback:]
X_ = X_.reshape(1, lookback, 1)

y_ = model.predict(X_).reshape(-1,1)
y_ = sc.inverse_transform(y_)

dataset_past = dataset[['Close']][:-30].reset_index()
dataset_past.rename(columns={'index':'Date','Close':'Actual'},inplace=True)
dataset_past['Date'] = pd.to_datetime(dataset_past['Date'])
dataset_past['Forecast'] = np.nan
dataset_past['Forecast'].iloc[-1] = dataset_past['Actual'].iloc[-1]

dataset_future = pd.DataFrame(columns=['Date','Actual','Forecast'])
dataset_future['Date'] = pd.date_range(start=dataset_past['Date'].iloc[-1] + pd.Timedelta(days=1), periods = forecast)
dataset_future['Actual'] = np.nan
dataset_future['Forecast'] = y_.flatten()
dataset_past, dataset_future

actual_preds = pd.concat([dataset_past.iloc[-100:], dataset_future], ignore_index=True)
actual_preds=actual_preds.set_index('Date')
actual_preds

In [ ]:
actual_preds.plot(title='TSLA')
print(actual_preds[-forecast:].drop('Actual',axis='columns'))

In [ ]:
dataset_future_actual = dataset[['Close']][-30:].reset_index()
dataset_future_actual.rename(columns={'index':'Date','Close':'30day History'},inplace=True)
dataset_future_actual['Date'] = pd.to_datetime(dataset_future_actual['Date'])
dataset_future_actual['Forecast'] = np.nan
dataset_future_actual = dataset_future_actual.set_index('Date')
dataset_future_actual = dataset_future_actual.drop('Forecast',axis='columns')
dataset_future_actual['Forecast'] = y_.flatten()
dataset_future_actual.plot(title='TSLA actual 30 day history')